In [3]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-12 09:55 ajna         INFO     Configuração de log efetuada


2025-09-12 09:55:18,105 ajna         INFO     Configuração de log efetuada


In [22]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [24]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores
LISTA_REDEX = "('8931309', '8933204')"  # Redex locais específicos

from datetime import datetime, timedelta

agora = datetime.now()
ha_7_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_7_dias.strftime('%Y-%m-%d %H:%M:%S')

In [25]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
"""

In [26]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""

In [27]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_REDEX}
"""

In [36]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND codigoRecinto IN {LISTA_REDEX}
"""

In [37]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [38]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [39]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao


In [46]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='left', on=['placa'], suffixes=('_E_Operador', '_S_REDEX'))

In [47]:
df_saidas_entradas

,id_E_Operador,placa,numeroConteiner_E_Operador,codigoRecinto_E_Operador,dataHoraOcorrencia_E_Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,numeroConteiner_S_REDEX,codigoRecinto_S_REDEX,dataHoraOcorrencia_S_REDEX,direcao_S_REDEX
0,12186563,GYJ1195,FTAU2057659,8931318,2025-09-08 00:11:58,C,E,01896217000194,51215393806,ANDREW COSTA DOS SANTOS SILVA,,152505264448668,NaN,NaN,NaN,NaT,NaN
1,12186564,DJF1E55,,8931318,2025-09-08 00:22:37,C,E,08900071000152,27644470867,JORGE DOS SANTOS SILVA,,,NaN,NaN,NaN,NaT,NaN
2,12186565,DBM7C63,MSMU2245591,8931318,2025-09-08 00:24:48,C,E,01896217000194,26571008807,PAULO EDUARDO FIDELIS,,,NaN,NaN,NaN,NaT,NaN
3,12186569,DPE7E30,BGBU5134130,8931318,2025-09-08 00:32:06,C,E,12014047000193,63262126691,AMILTON GONCALVES AMANCIO,,152505261149040,NaN,NaN,NaN,NaT,NaN
4,12186570,BWN6E31,UETU5852906,8931318,2025-09-08 00:35:20,C,E,01896217000194,10848936817,JOSE BENJAMIN DA SILVA,,,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28644,12296748,GIA6H99,,8931404,2025-09-11 23:52:05,C,E,61102778000287,,ROBERT GABRIEL COFFANI CRESPO,,,NaN,NaN,NaN,NaT,NaN
28645,12296752,MTN9C73,TGBU3928734,8931404,2025-09-11 23:52:14,C,E,43388833000184,41116258862,FABIO HENRIQUE DE SOUZA SANTOS,,,NaN,NaN,NaN,NaT,NaN
28646,12296751,CUA6E84,,8931404,2025-09-11 23:52:37,C,E,45050663000159,31455878898,ALEXSANDRO BARBOSA DA CRUZ,,,NaN,NaN,NaN,NaT,NaN
28647,12296755,MFO5H96,,8931404,2025-09-11 23:53:52,C,E,01040049000130,04796650601,NEILTON SANTANA DE SÃ?,,,NaN,NaN,NaN,NaT,NaN


In [ ]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 2) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 2) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

NameError: name 'engine' is not defined